In [1]:
import pandas as pd
import numpy as np
import datetime
from scorepi import *
from epiweeks import Week
from collections import defaultdict
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option("display.precision", 2)

## Configurations

In [2]:
rd = 5
start_week = Week(2021,18)
end_week = Week(2021,43)
max_date = datetime.datetime(2021, 6, 23)
models = ["Ensemble","Ensemble_LOP","JHUAPL-Bucky","JHU_IDD-CovidSP","Karlen-pypm",
          "MOBS_NEU-GLEAM_COVID","UNCC-hierbin","USC-SIkJalpha","UVA-adaptive"]

In [3]:
# rd = 12
# start_week = Week(2022,2)
# end_week = Week(2022,13)
# max_date = datetime.datetime(2022, 3, 15)
# models = ["Ensemble","Ensemble_LOP","Ensemble_LOP_untrimmed","JHU_IDD-CovidSP","MOBS_NEU-GLEAM_COVID","NCSU-COVSIM","NotreDame-FRED","UNCC-hierbin","USC-SIkJalpha",
#           "UTA-ImmunoSEIRS","UVA-EpiHiper","UVA-adaptive"]

In [4]:
# rd = 14
# start_week = Week(2022,23)
# end_week = Week(2023,22)
# max_date = None
# models = ["Ensemble","Ensemble_LOP","Ensemble_LOP_untrimmed","JHU_IDD-CovidSP","MOBS_NEU-GLEAM_COVID","MOBS_NEU-GLEAM_COVID_OT","NCSU-COVSIM",
#           "UNCC-hierbin","USC-SIkJalpha","USC-SIkJalpha-update","UTA-ImmunoSEIRS","UVA-adaptive"]

In [17]:
target = 'death'
#target = 'hosp'
incidence = True
target_location = '37'

locations = pd.read_csv('./dat/locations.csv',dtype={'location':str})
location_mapping = dict()
for loc in locations['location']:
    location_mapping[loc] = locations[locations['location'] == loc]['abbreviation'].unique()[0]

## Load and format surveillance data

In [18]:
if target == 'hosp':
    target_obs = 'hospitalization'
else:
    target_obs = target
observations = pd.read_parquet(f"./dat/truth_{'inc' if incidence else 'cum'}_{target_obs}.pq")
observations['date'] = pd.to_datetime(observations['date'])

#filter start - end week
observations = observations[(observations['date'] >= pd.to_datetime(start_week.startdate())) & 
                            (observations['date'] <= pd.to_datetime(end_week.enddate()))]


    
#filter location
observations = observations[observations['location'] == target_location]

#aggregate to weekly
observations = observations.groupby(['location', pd.Grouper(key='date', freq='W-SAT')]).sum().reset_index()

if max_date:
    observations = observations[observations['date'] <= max_date].copy()

#transform to Observation object
observations = Observations(observations)

/var/folders/1k/msfckb6d40q9g5y9b70y58hr0000gn/T/ipykernel_37458/177555968.py:18: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  observations = observations.groupby(['location', pd.Grouper(key='date', freq='W-SAT')]).sum().reset_index()


In [19]:
observations

,location,date,value
0,37,2021-05-08,129
1,37,2021-05-15,82
2,37,2021-05-22,96
3,37,2021-05-29,120
4,37,2021-06-05,73
5,37,2021-06-12,114
6,37,2021-06-19,75


## Load predictions, filter and score each model

In [20]:
target_prediction_list = [f"{i} wk ahead {'inc' if incidence else 'cum'} {target}" for i in range(1,len(observations)+1)]

If not well conditionned (quantile incoherent) we just skip the model

In [21]:
def check_coherence(predictions): 
    quantiles = predictions['quantile'].dropna().unique()
    qlist = quantiles[quantiles < 0.5]
    qlist_ = list(reversed(quantiles[quantiles > 0.5]))
    for i,q in enumerate(qlist):
        low = predictions[(predictions['quantile'] == q) & (predictions['location'] == 'US')].sort_values(
            by=['target_end_date'])
        up = predictions[(predictions['quantile'] == qlist_[i]) & (predictions['location'] == 'US')].sort_values(
            by=['target_end_date'])
        if not all((up['value'].to_numpy() - low['value'].to_numpy()) > 0):
            raise RuntimeError(f"quantile {q} and {qlist_[i]} not well conditioned")

In [22]:
agg_data_df_list = []
for model in models:    
    try:
        predictions = pd.read_parquet(f'./dat/{model}_rd{rd}.pq')
        predictions['target_end_date'] = pd.to_datetime(predictions['target_end_date'])
        predictions = predictions.merge(locations, how='inner', on='location')
        
        #choose location
        pred = predictions[predictions['location'] == target_location].copy()
        
        #filter target
        pred = pred[(pred['target'].isin(target_prediction_list))]
            
        #filter max date
        if max_date:
            pred = pred[pred['target_end_date'] <= max_date]
            
        if len(pred) == 0:
            raise RuntimeError(f"There are no predictions for model {model} at location {target_location}")
        
        #check if incoherent quantile, remove model
        check_coherence(pred)

        scenarios = list(pred['scenario_id'].drop_duplicates())
        predictions_list = [Predictions(pred[pred['scenario_id'] == scenario], t_col='target_end_date') for scenario in scenarios]
        med_ensemble_predictions = median_ensemble(predictions_list)
        ex_ensemble_predictions = extreme_ensemble(predictions_list)
        predictions_list += [med_ensemble_predictions, ex_ensemble_predictions]
        labels = ["Scenario " + scenario[0] for scenario in scenarios] + ["Median aggregate", "Extreme aggregate"]

        aggregated_scores = dict()
        for label,predictions in zip(labels,predictions_list):
            d,_ = score_utils.all_scores_from_df(observations, predictions, mismatched_allowed=False)
            aggregated_scores[label] = d

        aggregated_data = defaultdict(list)
        for label in labels:
            aggregated_data['Case'] += [label]
            aggregated_data['WIS'] += [aggregated_scores[label]['wis_mean']]

        agg_data_df = pd.DataFrame(aggregated_data)
        agg_data_df['Model'] = model
        agg_data_df_list.append(agg_data_df)
    except Exception as e:
        print(e)
        print(model)

/Users/clara/Documents/GitHub/scorepi/scorepi/ensemble_models.py:32: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ensemble_predictions = all_predictions.groupby(
/Users/clara/Documents/GitHub/scorepi/scorepi/ensemble_models.py:63: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ensemble_predictions_med = all_predictions[np.isclose(all_predictions[quantile_col],0.5)].groupby(
/Users/clara/Documents/GitHub/scorepi/scorepi/ensemble_models.py:65: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify n

[Errno 2] No such file or directory: './dat/UNCC-hierbin_rd5.pq'
UNCC-hierbin


/Users/clara/Documents/GitHub/scorepi/scorepi/ensemble_models.py:32: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ensemble_predictions = all_predictions.groupby(
/Users/clara/Documents/GitHub/scorepi/scorepi/ensemble_models.py:63: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ensemble_predictions_med = all_predictions[np.isclose(all_predictions[quantile_col],0.5)].groupby(
/Users/clara/Documents/GitHub/scorepi/scorepi/ensemble_models.py:65: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify n

In [23]:
agg_data_df = pd.concat(agg_data_df_list)
agg_data_df = agg_data_df.pivot(index=['Model'], columns=['Case'])
# agg_data_df[('WIS','Scenario average')] = agg_data_df['WIS'][['Scenario A','Scenario B','Scenario C','Scenario D']].mean(axis=1)
agg_data_df = agg_data_df.reindex(columns=[('WIS','Median aggregate'),
                                           ('WIS','Extreme aggregate'),
                                           # ('WIS','Scenario average'),
                                           ('WIS','Scenario A'),
                                           ('WIS','Scenario B'),
                                           ('WIS','Scenario C'),
                                           ('WIS','Scenario D')])
minval = agg_data_df.to_numpy().min()
agg_data_df = agg_data_df.apply(lambda val: val/minval)
agg_data_df = agg_data_df.join(agg_data_df.rank(method='min').astype(int).rename(columns={'WIS':'Rank'}))
agg_data_df = agg_data_df.rename(columns={'WIS':'Relative WIS'})
agg_data_df = agg_data_df.sort_values(by=[('Rank','Median aggregate')])

In [24]:
def make_pretty(styler):
    styler.format(precision=2)
    # styler.format_index(lambda v: v.strftime("%A"))
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="Blues")
    return styler
def highlight_min(s, props=''):
    return np.where(s == np.nanmin(s.values), props, '')
def highlight_max(s, props=''):
    return np.where(s == np.nanmax(s.values), props, '')

In [25]:
target_loc_name = location_mapping[target_location]

pandas.core.frame.DataFrame

In [26]:
s = agg_data_df['Relative WIS'].style.pipe(make_pretty)
#s = agg_data_df['Relative WIS']
s = s.apply(highlight_min, props='color:green', axis=0)
# s = s.apply(highlight_max, props='color:', axis=0)
s = s.set_caption(f"Relative WIS, Round {rd}, target = {target_obs}, location = {target_loc_name}")

In [27]:
caption = {'selector': 'caption','props': 'caption-side: top; font-size:1.5em;'}

cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: grey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: lightgrey; color: #1a1a1a;'
}
values = {'selector': 'td', 'props': 'text-align: right;'}
s.set_table_styles([caption,cell_hover, index_names, headers, values])
s

Case,Median aggregate,Extreme aggregate,Scenario A,Scenario B,Scenario C,Scenario D
Model,,,,,,
Ensemble,1.01,1.12,1.09,1.04,1.00,1.07
Ensemble_LOP,1.04,1.17,1.09,1.07,1.03,1.11
MOBS_NEU-GLEAM_COVID,1.13,1.22,1.34,1.11,1.09,1.15
USC-SIkJalpha,1.58,1.63,1.54,1.55,1.61,1.74
JHUAPL-Bucky,1.70,1.75,1.68,1.73,1.68,1.73
UVA-adaptive,1.97,2.00,1.74,2.23,1.74,2.25
Karlen-pypm,2.75,2.63,2.00,2.30,3.25,3.44
JHU_IDD-CovidSP,3.15,3.12,3.15,3.09,3.17,3.22


In [18]:
import imgkit
from pathlib import Path
path = f"./figs/rd{rd}/{'inc' if incidence else 'cum'}_{target}/{target_loc_name}/"
Path(path).mkdir(parents=True, exist_ok=True)
imgkit.from_string(s.to_html(), path + 'relative_WIS.png', options = {
        'zoom': 4,
        'width': int(1920*1.8), 
        'quality': 100})

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


True